In [ ]:
import numpy as np

In [ ]:
# UVD imputer
def UVD(X, lowRank, regParam):
    
    # parameters and random initialization
    maxIter = 200; eps = 1e-5; n, m = X.shape
    U = np.random.rand(n, lowRank) / lowRank
    V = np.random.rand(m, lowRank) / lowRank
    
    # binary matrix of observed and not observed values
    B = ~np.isnan(X); B = B.astype(np.float64, copy=False)
    
    # ALS algorithm
    X = np.nan_to_num(X, copy=True)
    RMSE = [np.linalg.norm(B * (X - np.matmul(U, V.T)), 'fro') / np.sqrt(np.sum(B))]
    for iter in range(maxIter):
        # update U
        for i, bi in enumerate(B):
            U[i] = np.linalg.solve(np.matmul(V.T, np.matmul(np.diag(bi), V)) + regParam * np.eye(lowRank), 
                                   np.matmul(V.T, X[i].T)).T
        # update V
        for j, bj in enumerate(B.T):
            V[j] = np.linalg.solve(np.matmul(U.T, np.matmul(np.diag(bj), U)) + regParam * np.eye(lowRank), 
                                   np.matmul(U.T, X[:, j])).T
        # stop criterion
        RMSE.append(np.linalg.norm(B * (X - np.matmul(U, V.T)), 'fro') / np.sqrt(np.sum(B)))
        if abs(RMSE[iter+1]-RMSE[iter]) <= eps:
            break
    # imputed matrix
    X_hat = np.matmul(U, V.T)
    
    return X_hat, U, V